In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.model_selection import KFold
xgb.__version__

'1.6.1'

In [2]:
colleges = pd.read_csv('../../input/colleges/colleges.csv')
colleges.head()

UNITID                            school_name        city state  \
0  100654             'Alabama A & M University'      Normal    AL   
1  100663  'University of Alabama at Birmingham'  Birmingham    AL   
2  100690                   'Amridge University'  Montgomery    AL   
3  100706  'University of Alabama in Huntsville'  Huntsville    AL   
4  100724             'Alabama State University'  Montgomery    AL   

          zip                  school_webpage latitude           longitude  \
0       35762                   www.aamu.edu/  34.7834            -86.5685   
1  35294-0110                     www.uab.edu  33.5022            -86.8092   
2  36117-3553       www.amridgeuniversity.edu  32.3626  -86.17399999999999   
3       35899                     www.uah.edu  34.7228            -86.6384   
4  36104-0271  www.alasu.edu/email/index.aspx  32.3643            -86.2957   

  admission_rate sat_verbal_midrange  ...  \
0         0.8989               410.0  ...   
1         0.8673               580.0  ...   
2              ?                   ?  ...   
3         0.8062               575.0  ...   
4         0.5125               430.0  ...   

                              carnegie_undergraduate  \
0                    'Full-time four-year inclusive'   
1  'Medium full-time four-year selective higher t...   
2  'Medium full-time four-year inclusivestudents ...   
3  'Medium full-time four-year selective higher t...   
4                    'Full-time four-year inclusive'   

                                       carnegie_size religious_affiliation  \
0  'Medium 4-year highly residential (3000 to 9999)'                     ?   
1  'Large 4-year primarily nonresidential (over 9...                     ?   
2  'Very small 4-year primarily nonresidential (l...  'Churches of Christ'   
3  'Medium 4-year primarily nonresidential (3000 ...                     ?   
4  'Medium 4-year primarily residential (3000 to ...                     ?   

     percent_female           agege24    faminc mean_earnings_6_years  \
0  0.52999997138977  0.07999999821186  40211.22               26100.0   
1  0.64999997615814  0.25999999046325  49894.65               37400.0   
2  0.50999999046325  0.82999998331069  38712.18               38500.0   
3  0.55000001192092  0.28999999165534   54155.4               39300.0   
4  0.56999999284744  0.10999999940395  31846.99               21200.0   

  median_earnings_6_years mean_earnings_10_years median_earnings_10_years  
0                 22800.0                35300.0                  31400.0  
1                 33200.0                46300.0                  40300.0  
2                 32800.0                42100.0                  38100.0  
3                 36700.0                52700.0                  46600.0  
4                 19300.0                30700.0                  27800.0  

[5 rows x 48 columns]

In [3]:
colleges.shape

(7063, 48)

In [4]:
colleges.columns

Index(['UNITID', 'school_name', 'city', 'state', 'zip', 'school_webpage',
       'latitude', 'longitude', 'admission_rate', 'sat_verbal_midrange',
       'sat_math_midrange', 'sat_writing_midrange', 'act_combined_midrange',
       'act_english_midrange', 'act_math_midrange', 'act_writing_midrange',
       'sat_total_average', 'undergrad_size', 'percent_white', 'percent_black',
       'percent_hispanic', 'percent_asian', 'percent_part_time',
       'average_cost_academic_year', 'average_cost_program_year',
       'tuition_(instate)', 'tuition_(out_of_state)', 'spend_per_student',
       'faculty_salary', 'percent_part_time_faculty', 'percent_pell_grant',
       'completion_rate', 'predominant_degree', 'highest_degree', 'ownership',
       'region', 'gender', 'carnegie_basic_classification',
       'carnegie_undergraduate', 'carnegie_size', 'religious_affiliation',
       'percent_female', 'agege24', 'faminc', 'mean_earnings_6_years',
       'median_earnings_6_years', 'mean_earnings_10_y

In [5]:
colleges['admission_rate'] = pd.to_numeric(colleges['admission_rate'], errors='coerce')
colleges['undergrad_size'] = pd.to_numeric(colleges['undergrad_size'], errors='coerce')
colleges['sat_math_midrange'] = pd.to_numeric(colleges['sat_math_midrange'], errors='coerce')
colleges['sat_verbal_midrange'] = pd.to_numeric(colleges['sat_verbal_midrange'], errors='coerce')
colleges['act_writing_midrange'] = pd.to_numeric(colleges['act_writing_midrange'], errors='coerce')
colleges['act_math_midrange'] = pd.to_numeric(colleges['act_math_midrange'], errors='coerce')
colleges['act_english_midrange'] = pd.to_numeric(colleges['act_english_midrange'], errors='coerce')
colleges['act_combined_midrange'] = pd.to_numeric(colleges['act_combined_midrange'], errors='coerce')
colleges['completion_rate'] = pd.to_numeric(colleges['completion_rate'], errors='coerce') 
colleges['tuition_(instate)'] = pd.to_numeric(colleges['tuition_(instate)'], errors='coerce')
colleges['tuition_(out_of_state)'] = pd.to_numeric(colleges['tuition_(out_of_state)'], errors='coerce')
colleges['percent_white'] = pd.to_numeric(colleges['percent_white'], errors='coerce')
colleges['percent_black'] = pd.to_numeric(colleges['percent_black'], errors='coerce')
colleges['percent_hispanic'] = pd.to_numeric(colleges['percent_hispanic'], errors='coerce')
colleges['percent_asian'] = pd.to_numeric(colleges['percent_asian'], errors='coerce')
colleges['sat_total_average'] = pd.to_numeric(colleges['sat_total_average'], errors='coerce')
colleges['completion_rate'] = pd.to_numeric(colleges['completion_rate'], errors='coerce')
colleges['percent_female'] = pd.to_numeric(colleges['percent_female'], errors='coerce')
colleges['agege24'] = pd.to_numeric(colleges['agege24'], errors='coerce')
colleges['faculty_salary'] = pd.to_numeric(colleges['faculty_salary'], errors='coerce')
colleges['percent_part_time'] = pd.to_numeric(colleges['percent_part_time'], errors='coerce')
colleges['faminc'] = pd.to_numeric(colleges['faminc'], errors='coerce')
colleges['average_cost_academic_year'] = pd.to_numeric(colleges['average_cost_academic_year'], errors='coerce')
colleges['average_cost_program_year'] = pd.to_numeric(colleges['average_cost_program_year'], errors='coerce')
colleges['spend_per_student'] = pd.to_numeric(colleges['spend_per_student'], errors='coerce')
colleges['mean_earnings_6_years'] = pd.to_numeric(colleges['mean_earnings_6_years'], errors='coerce')
colleges['median_earnings_6_years'] = pd.to_numeric(colleges['median_earnings_6_years'], errors='coerce')
colleges['mean_earnings_10_years'] = pd.to_numeric(colleges['mean_earnings_10_years'], errors='coerce')
colleges['median_earnings_10_years'] = pd.to_numeric(colleges['median_earnings_10_years'], errors='coerce')
colleges['latitude'] = pd.to_numeric(colleges['latitude'], errors='coerce')
colleges['longitude'] = pd.to_numeric(colleges['longitude'], errors='coerce')

In [6]:
columns = ['undergrad_size', 'spend_per_student', 'admission_rate', 'mean_earnings_6_years', 'median_earnings_6_years', 'sat_math_midrange', 'sat_verbal_midrange', 
           'act_math_midrange', 'act_writing_midrange', 'latitude', 'longitude', 'completion_rate', 'tuition_(out_of_state)', 'tuition_(instate)', 'percent_white', 
           'percent_black', 'percent_hispanic', 'percent_asian', 'median_earnings_10_years', 'sat_total_average', 'completion_rate', 'faculty_salary', 'percent_female',
          'percent_part_time', 'agege24', 'faminc', 'average_cost_program_year', 'average_cost_academic_year']

In [7]:
target = colleges['percent_pell_grant'].values

In [147]:
errors = []
kf = KFold(n_splits=5, random_state=137, shuffle=True)
for train_index, test_index in kf.split(colleges):
    X_train, X_test = colleges.loc[train_index], colleges.loc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    clf = XGBRegressor(n_estimators=3500, learning_rate=0.01, max_depth=6, subsample=0.9, colsample_bytree=0.95, colsample_bynode=0.8,
                       tree_method='gpu_hist', min_child_weight=2, reg_alpha=0.01)
    clf.fit(X_train[columns].values, y_train)
    preds = clf.predict(X_test[columns].values)
    error = mean_squared_error(y_test, preds, squared=True)
    print("Fold mean squared error:", error)
    errors.append(error)

Fold mean squared error: 0.020419346365676348
Fold mean squared error: 0.019853995266933905
Fold mean squared error: 0.02029785433509362
Fold mean squared error: 0.018564315836440956
Fold mean squared error: 0.020014994917144036


In [148]:
1000*np.mean(errors)

19.83010134425777

In [149]:
1000*np.std(errors)

0.663772861803951